In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mysql.connector as con

In [2]:
class Connection:
    @staticmethod
    def getConnection():
        db_connection = con.connect(                 #Creating Connection
            host="localhost",
            user="root",
            password="alpha",
            database="electricity_billing_db"
        )
        return db_connection

In [3]:
class Customer:
    def __init__(self,uid,uname,pin,fname,address,role):
        self.uid=uid
        self.uname=uname
        self.pin=pin
        self.fname=fname
        self.address=address
        self.role=role        

In [4]:
class Admin:
    def __init__(self,uid,uname,pin,fname,address,role):
        self.uid=uid
        self.uname=uname
        self.pin=pin
        self.fname=fname
        self.address=address
        self.role=role    

In [11]:
class CheckUser:
    def get_user(username):
        db_con=Connection.getConnection()
        cursor = db_con.cursor(dictionary=True)
        cursor.execute("SELECT * from users WHERE username=%s",(username,))
        user=cursor.fetchone()
        if user:
            if user["role"]=='customer':
                return Customer(user["user_id"],user["username"],user["password"],user["full_name"],user["address"],user["role"])
            else:
                return Admin(user["user_id"],user["username"],user["password"],user["full_name"],user["address"],user["role"])
        else:
            print("User Not Found!")

In [19]:
user=CheckUser.get_user("user_50") 
print(user.fname)

User 50
